In [1]:
# ============================================================
# 01_data_lake_exploration.ipynb
# Khám phá và lưu dữ liệu gốc MovieLens + IMDb + Tag Genome 2021
# ============================================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os, glob, shutil

# ------------------------------------------------------------
# 1️⃣ KHỞI TẠO SPARK SESSION
# ------------------------------------------------------------
spark = SparkSession.builder \
    .appName("MovieLens_DataLake_Exploration") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("✅ SparkSession khởi tạo thành công!")

✅ SparkSession khởi tạo thành công!


In [9]:
# ------------------------------------------------------------
# 2️⃣ KHAI BÁO ĐƯỜNG DẪN
# ------------------------------------------------------------
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

RAW_PATH = os.path.join(BASE_DIR, "data/raw/")
DATALAKE_PATH = os.path.join(BASE_DIR, "data/datalake/")

os.makedirs(DATALAKE_PATH, exist_ok=True)

print("RAW_PATH:", RAW_PATH)
print("DATALAKE_PATH:", DATALAKE_PATH)

RAW_PATH: /home/jovyan/data/raw/
DATALAKE_PATH: /home/jovyan/data/datalake/


In [3]:
# ------------------------------------------------------------
# 3️⃣ HÀM ĐỌC JSON TỰ ĐỘNG (XỬ LÝ 1 DÒNG HOẶC NHIỀU DÒNG)
# ------------------------------------------------------------
def read_json_auto(path):
    df = spark.read.json(path)
    if df.count() == 1 and len(df.columns) == 1:
        col_name = df.columns[0]
        df = df.select(F.explode(F.col(col_name)).alias("data")).select("data.*")
    return df

In [10]:
# ------------------------------------------------------------
# 4️⃣ ĐỌC 6 FILE JSON GỐC
# ------------------------------------------------------------
metadata = read_json_auto(RAW_PATH + "metadata.json")
ratings = read_json_auto(RAW_PATH + "ratings.json")
reviews = read_json_auto(RAW_PATH + "reviews.json")
survey_answers = read_json_auto(RAW_PATH + "survey_answers.json")
tag_count = read_json_auto(RAW_PATH + "tag_count.json")
tags = read_json_auto(RAW_PATH + "tags.json")

print("✅ Hoàn tất đọc dữ liệu gốc!")

✅ Hoàn tất đọc dữ liệu gốc!


In [11]:
# ------------------------------------------------------------
# 5️⃣ THỐNG KÊ SƠ BỘ
# ------------------------------------------------------------
print("📊 Số dòng mỗi bảng:")
print("metadata:", metadata.count())
print("ratings:", ratings.count())
print("reviews:", reviews.count())
print("survey_answers:", survey_answers.count())
print("tag_count:", tag_count.count())
print("tags:", tags.count())

📊 Số dòng mỗi bảng:
metadata: 84661
ratings: 28490116
reviews: 2624608
survey_answers: 58903
tag_count: 212704
tags: 1094


In [12]:
# ------------------------------------------------------------
# 6️⃣ KHÁM PHÁ SCHEMA
# ------------------------------------------------------------
print("\n🔍 Schema metadata:")
metadata.printSchema()

print("\n🔍 Schema ratings:")
ratings.printSchema()

print("\n🔍 Schema reviews:")
reviews.printSchema()


🔍 Schema metadata:
root
 |-- avgRating: double (nullable = true)
 |-- dateAdded: string (nullable = true)
 |-- directedBy: string (nullable = true)
 |-- imdbId: string (nullable = true)
 |-- item_id: long (nullable = true)
 |-- starring: string (nullable = true)
 |-- title: string (nullable = true)


🔍 Schema ratings:
root
 |-- item_id: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- user_id: long (nullable = true)


🔍 Schema reviews:
root
 |-- item_id: long (nullable = true)
 |-- txt: string (nullable = true)



In [13]:
# ------------------------------------------------------------
# 7️⃣ XEM MẪU DỮ LIỆU (TOP 3 DÒNG)
# ------------------------------------------------------------
print("\n👀 Sample metadata:")
metadata.show(3, truncate=False)

print("\n👀 Sample ratings:")
ratings.show(3, truncate=False)

print("\n👀 Sample tags:")
tags.show(3, truncate=False)


👀 Sample metadata:
+---------+---------+-------------+-------+-------+---------------------------------------------------------------------------------------------------------------------------------------+-----------------------+
|avgRating|dateAdded|directedBy   |imdbId |item_id|starring                                                                                                                               |title                  |
+---------+---------+-------------+-------+-------+---------------------------------------------------------------------------------------------------------------------------------------+-----------------------+
|3.89146  |NULL     |John Lasseter|0114709|1      |Tim Allen, Tom Hanks, Don Rickles, Jim Varney, John Ratzenberger, Wallace Shawn, Laurie Metcalf, John Morris, R. Lee Ermey, Annie Potts|Toy Story (1995)       |
|3.26605  |NULL     |Joe Johnston |0113497|2      |Jonathan Hyde, Bradley Pierce, Robin Williams, Kirsten Dunst                     

In [18]:
# ------------------------------------------------------------
# 8️⃣ HÀM TIỆN ÍCH LƯU CSV (1 FILE DUY NHẤT)
# ------------------------------------------------------------
def save_single_csv(df, name):
    tmp_dir = os.path.join(DATALAKE_PATH, f"{name}_tmp")
    final_path = os.path.join(DATALAKE_PATH, f"{name}.csv")

    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(tmp_dir)
    part_file = glob.glob(os.path.join(tmp_dir, "part-*.csv"))[0]
    shutil.move(part_file, final_path)
    shutil.rmtree(tmp_dir)

    print(f"✅ Đã lưu {name}.csv → {final_path} ({df.count():,} dòng)")

In [19]:
# ------------------------------------------------------------
# 9️⃣ LƯU BẢNG THÀNH CSV (RAW DATA → DATA LAKE)
# ------------------------------------------------------------
save_single_csv(metadata, "metadata")
save_single_csv(ratings, "ratings")
save_single_csv(reviews, "reviews")
save_single_csv(survey_answers, "survey_answers")
save_single_csv(tag_count, "tag_count")
save_single_csv(tags, "tags")

✅ Đã lưu metadata.csv → /home/jovyan/data/datalake/metadata.csv (84,661 dòng)
✅ Đã lưu ratings.csv → /home/jovyan/data/datalake/ratings.csv (28,490,116 dòng)
✅ Đã lưu reviews.csv → /home/jovyan/data/datalake/reviews.csv (2,624,608 dòng)
✅ Đã lưu survey_answers.csv → /home/jovyan/data/datalake/survey_answers.csv (58,903 dòng)
✅ Đã lưu tag_count.csv → /home/jovyan/data/datalake/tag_count.csv (212,704 dòng)
✅ Đã lưu tags.csv → /home/jovyan/data/datalake/tags.csv (1,094 dòng)


In [21]:
# ------------------------------------------------------------
# 🔟 KIỂM TRA FILE ĐÃ LƯU
# ------------------------------------------------------------
print("\n📂 Danh sách file trong data/datalake:")
for file in os.listdir(DATALAKE_PATH):
    file_path = os.path.join(DATALAKE_PATH, file)
    
    # Bỏ qua thư mục con hoặc file trống
    if not os.path.isfile(file_path):
        continue
    
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f" - {file:25} {size_mb:8.2f} MB")

print("\n🎯 DONE: Khám phá + lưu dữ liệu RAW → DATA LAKE thành công!")


📂 Danh sách file trong data/datalake:
 - tag_count.csv                 2.84 MB
 - reviews.csv                3688.28 MB
 - tags.csv                      0.02 MB
 - ratings.csv                 432.31 MB
 - metadata.csv                 11.84 MB
 - survey_answers.csv            1.12 MB

🎯 DONE: Khám phá + lưu dữ liệu RAW → DATA LAKE thành công!
